In [27]:
import pandas as pd
import re
import requests
from tqdm import tqdm
import time
from datetime import datetime, timedelta
from io import StringIO

In [30]:
# A python class to download historical stock prices in csv files.
# It makes two requests, one to get the cookie and the crumb and another one to get the data.
# Source : https://stackoverflow.com/questions/44225771/scraping-historical-data-from-yahoo-finance-with-python

class YahooFinanceHistory:
    timeout = 5
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [31]:
# Sample Data
df = YahooFinanceHistory('ICICIBANK.NS', days_back=30).get_quote()
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-10-19,402.049988,418.750000,401.149994,417.100006,417.100006,41072871
1,2020-10-20,413.500000,416.899994,411.000000,414.600006,414.600006,26158669
2,2020-10-21,416.000000,428.500000,410.299988,420.299988,420.299988,43124966
3,2020-10-22,416.399994,419.649994,407.649994,412.899994,412.899994,33088074
4,2020-10-23,417.000000,420.500000,411.750000,416.950012,416.950012,22151463


In [32]:
# Import Lookup Data
nifty50_lookup = pd.read_csv("nifty50_lookuptable.csv")
nifty50_lookup.head()

,Sr.No.,Company Name,Sector,Weightage,thehindu_searchstring,mcontrol_substring,ticker,yfin_ticker
0,1,Reliance Industries Ltd.,Petroleum Products,14.93%,reliance%20petroleum,RI,RELIANCE,RELIANCE
1,2,HDFC Bank Ltd.,Banks,9.69%,hdfc%20bank,HDF01,HDFCBANK,HDFCBANK
2,3,Infosys Limited,Software,7.63%,infosys,IT,INFY,INFY
3,4,Housing Development Fin. Corp. Ltd.,Finance,6.44%,hdfc,HDF,HDFC,HDFC
4,5,Tata Consultancy Services Ltd.,Software,5.41%,tcs,TCS,TCS,TCS


In [13]:
# Sample scraping to validate yfin_ticker
for tkr in tqdm(nifty50_lookup['yfin_ticker']):
    df = YahooFinanceHistory(tkr+'.NS').get_quote()

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:47<00:00,  2.15s/it]


In [33]:
# Scraping for past 'yr' number of years
yr = 3
d = yr*365
for tkr in tqdm(nifty50_lookup['yfin_ticker']):
    df = YahooFinanceHistory(tkr+'.NS', days_back=d).get_quote()
    print("Stock Prices Extracted for Past {} Years for {}".format(yr, tkr))
    print("Total Usable Scraped Data :",df.shape)
    filename = "stocks_yfinance_" + tkr + "_3yrs" + ".pkl"
    df.to_pickle(filename)
    print("Oldest Available Article :", min(df['Date']).date())
    print("Data saved for {} : {}".format(tkr, filename))
    time.sleep(5)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Stock Prices Extracted for Past 3 Years for RELIANCE
Total Usable Scraped Data : (738, 7)
Oldest Available Article : 2017-11-20
Data saved for RELIANCE : stocks_yfinance_RELIANCE_3yrs.pkl


  2%|█▋                                                                                 | 1/50 [00:07<05:46,  7.07s/it]

Stock Prices Extracted for Past 3 Years for HDFCBANK
Total Usable Scraped Data : (738, 7)
Oldest Available Article : 2017-11-20
Data saved for HDFCBANK : stocks_yfinance_HDFCBANK_3yrs.pkl


  4%|███▎                                                                               | 2/50 [00:14<05:41,  7.11s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [05:51<00:14,  7.33s/it]

Stock Prices Extracted for Past 3 Years for IOC
Total Usable Scraped Data : (738, 7)
Oldest Available Article : 2017-11-20
Data saved for IOC : stocks_yfinance_IOC_3yrs.pkl


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [05:58<00:07,  7.24s/it]

Stock Prices Extracted for Past 3 Years for GAIL
Total Usable Scraped Data : (738, 7)
Oldest Available Article : 2017-11-20
Data saved for GAIL : stocks_yfinance_GAIL_3yrs.pkl


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [06:05<00:00,  7.32s/it]
